# EXPERIMENT TRACKING USING WandB

## https://wandb.ai/site/

In [1]:
# ! pip install wandb

In [2]:
! wandb login

wandb: Currently logged in as: amanjn2003 (amanjn2003-santa-clara-university). Use `wandb login --relogin` to force relogin


In [3]:
# Import Depedencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import datetime
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC
from tensorflow.keras.regularizers import L2
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback, WandbCallback

## Initializing a WandB run

In [4]:
wandb.init(name = "Run 1 - Inbuilt Callback", project="Malaria-Detection", entity="amanjn2003-santa-clara-university", settings=wandb.Settings(init_timeout=300))

wandb: Currently logged in as: amanjn2003 (amanjn2003-santa-clara-university). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [5]:
# Load Malaria Dataset From Tensorflow Datasets
dataset, datasetInfo = tfds.load("malaria", 
                                 with_info=True, 
                                 shuffle_files=True) # -----------++++++    ->    ---+++-++--+++-

# Dataset Split Function
def split(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
    trainDataset = dataset.take(int(TRAIN_RATIO*len(dataset)))
    leftDataset = dataset.skip(int(TRAIN_RATIO*len(dataset)))
    valDataset = leftDataset.take(int(VAL_RATIO*len(dataset)))
    testDataset = leftDataset.skip(int(VAL_RATIO*len(dataset)))
    return trainDataset, valDataset, testDataset

# Split Dataset As 80/10/10 
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
trainDataset, valDataset, testDataset = split(dataset['train'], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IMAGE_SIZE = 224
def resizeRescale(inputs):
    return tf.image.resize(inputs['image'], (IMAGE_SIZE, IMAGE_SIZE))/255., inputs['label']
trainDataset = trainDataset.map(resizeRescale)
valDataset = valDataset.map(resizeRescale)
testDataset = testDataset.map(resizeRescale)

# Dataset Batching
BATCH_SIZE=32
BUFFER_SIZE = 8
trainDataset = trainDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-01-27 11:46:38.138529: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-27 11:46:38.138557: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-27 11:46:38.138560: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-27 11:46:38.138576: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-27 11:46:38.138588: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## WandB Configurations

In [6]:
wandb.config = {
    "LEARNING_RATE" : 0.001,
    "N_EPOCHS" : 20,
    "BATCH_SIZE" : 128,
    "DROPOUT_RATE": 0.0,
    "IM_SIZE" : 224,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 128,
    "N_DENSE_2" : 32
}

In [7]:
# Using Sequential API
CONFIGURATION = wandb.config
IMAGE_SIZE = CONFIGURATION['IM_SIZE']
model = tf.keras.Sequential([
    Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),

    Conv2D(filters=CONFIGURATION["N_FILTERS"], kernel_size=CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"]),
    Dropout(rate = CONFIGURATION["DROPOUT_RATE"]),

    Conv2D(filters=CONFIGURATION["N_FILTERS"], kernel_size=CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"]),

    Flatten(),

    Dense(CONFIGURATION["N_DENSE_1"], activation='relu'),
    BatchNormalization(),
    Dropout(rate = CONFIGURATION["DROPOUT_RATE"]),

    Dense(CONFIGURATION["N_DENSE_2"], activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
])
model.summary()

# Model Metrics
Metrics = [BinaryAccuracy(), FalsePositives(), FalseNegatives(), TrueNegatives(), TruePositives(), Precision(), Recall(), AUC()]

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=Metrics
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 221, 221, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 221, 221, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 219, 219, 6)    │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 219, 219, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 218, 218, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 285144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    36,498,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,503,907 (139.25 MB)

 Trainable params: 36,503,563 (139.25 MB)

 Non-trainable params: 344 (1.34 KB)

## WandB Inbuilt Callbacks

In [8]:
model.fit(trainDataset, validation_data=valDataset, epochs=CONFIGURATION["N_EPOCHS"], verbose=1, callbacks=[WandbMetricsLogger(), WandbModelCheckpoint("Models/Model.keras")])

Epoch 1/20


2025-01-27 11:46:39.127023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


689/689 ━━━━━━━━━━━━━━━━━━━━ 77s 108ms/step - auc: 0.7200 - binary_accuracy: 0.6706 - false_negatives: 1415.7478 - false_positives: 2092.4827 - loss: 0.6392 - precision: 0.6568 - recall: 0.7231 - true_negatives: 3426.2449 - true_positives: 4121.4727 - val_auc: 0.7921 - val_binary_accuracy: 0.6759 - val_false_negatives: 93.0000 - val_false_positives: 800.0000 - val_loss: 0.6017 - val_precision: 0.6180 - val_recall: 0.9329 - val_true_negatives: 568.0000 - val_true_positives: 1294.0000
Epoch 2/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 73s 106ms/step - auc: 0.7933 - binary_accuracy: 0.7267 - false_negatives: 1127.3710 - false_positives: 1892.9072 - loss: 0.5752 - precision: 0.7016 - recall: 0.7960 - true_negatives: 3625.8752 - true_positives: 4409.7944 - val_auc: 0.7737 - val_binary_accuracy: 0.6537 - val_false_negatives: 67.0000 - val_false_positives: 887.0000 - val_loss: 0.6438 - val_precision: 0.5990 - val_recall: 0.9519 - val_true_negatives: 476.0000 - val_true_positives: 1325.0000
Epoch 3/20
68

In [9]:
wandb.run

## Check Dashboard Here
### https://wandb.ai/amanjn2003-santa-clara-university/Malaria-Detection?nw=nwuseramanjn2003

In [10]:
wandb.finish()

epoch/auc,▁▂▄▆▇▇▇▇████████████
epoch/binary_accuracy,▁▂▃▅▆▆▇▇▇▇▇▇████████
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/false_negatives,█▇▅▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/false_positives,█▇▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,██▇▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
epoch/precision,▁▂▃▄▅▆▆▇▇▇▇▇▇███████
epoch/recall,▁▂▄▆▇▇▇▇▇▇██████████
epoch/true_negatives,▁▂▃▄▅▆▆▇▇▇▇▇▇███████
epoch/true_positives,▁▂▄▆▇▇▇▇▇▇██████████


## Custom Callbacks With WandB

In [11]:
wandb.init(name="Run 2 - Custom Callbacks", project="Malaria-Detection", entity="amanjn2003-santa-clara-university", settings=wandb.Settings(init_timeout=300))

In [12]:
def getLabel(label):
    if label==0 or label==False:
        return "Parasitized"
    return "Unparasitized"

class ImageLoggingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Get model predictions
        val_images, val_labels = next(iter(valDataset))  # Get a batch of images and labels
        predictions = self.model.predict(val_images)
        
        # Convert predictions to class labels
        predicted_labels = np.array(predictions>=0.5)[:, 0]
        predicted_probabilities = [
            [1, 0] if predicted_labels[i] == 0 else [0, 1]
            for i in range(len(predicted_labels))
        ]        
        # Log images to Wandb with the predictions
        wandb.log({
            "epoch": epoch,
            "examples": [
                wandb.Image(val_images[i], caption=f"True: {getLabel(val_labels[i])}, Pred: {getLabel(predicted_labels[i])}")
                for i in range(min(5, len(val_images)))  # Log up to 5 images
            ],
            "ConfusionMatrix" : wandb.plot.confusion_matrix(probs=None, y_true=np.array(val_labels), preds=predicted_labels, class_names=["Parasitized", "Uninfected"]),
            "ROC-Curve" : wandb.plot.roc_curve(y_true=np.array(val_labels), y_probas=predicted_probabilities, labels=["Parasitized", "Uninfected"])
        })

In [13]:
model.fit(trainDataset, validation_data=valDataset, epochs=CONFIGURATION["N_EPOCHS"], verbose=1, callbacks=[WandbMetricsLogger(), WandbModelCheckpoint("Models/Model.keras"), ImageLoggingCallback()])

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/stepstep - auc: 0.9951 - binary_accuracy: 0.9752 - false_negatives: 98.8433 - false_positives: 184.5820 - loss: 0.1054 - precision: 0.9674 - recall: 0.9839 - true_negatives: 5326.2310 - true_positives: 5430.34
689/689 ━━━━━━━━━━━━━━━━━━━━ 76s 110ms/step - auc: 0.9951 - binary_accuracy: 0.9752 - false_negatives: 98.9942 - false_positives: 184.8420 - loss: 0.1054 - precision: 0.9674 - recall: 0.9839 - true_negatives: 5333.9595 - true_positives: 5438.1523 - val_auc: 0.9350 - val_binary_accuracy: 0.8915 - val_false_negatives: 232.0000 - val_false_positives: 67.0000 - val_loss: 0.9280 - val_precision: 0.9454 - val_recall: 0.8333 - val_true_negatives: 1296.0000 - val_true_positives: 1160.0000
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step/step - auc: 0.9958 - binary_accuracy: 0.9787 - false_negatives: 93.9013 - false_positives: 150.6401 - loss: 0.0987 - precision: 0.9745 - recall: 0.9833 - true_negatives: 5360.2046 - true_positives: 5435.251
689

## Check Dashboard Here
### https://wandb.ai/amanjn2003-santa-clara-university/Malaria-Detection?nw=nwuseramanjn2003

In [14]:
wandb.run

In [15]:
wandb.finish()

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/auc,▁▂▃▃▂▅▆▄▆▆▅▇▇▇█▇█▇▇█
epoch/binary_accuracy,▁▂▂▄▃▄▅▄▅▆▄▇▆▆▇▇▇██▇
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/false_negatives,█▇▇▆█▆▅▅▄▃▅▂▃▄▂▂▁▁▁▃
epoch/false_positives,█▆▆▅▅▄▄▅▄▃▄▂▂▂▂▂▂▁▁▂
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▆▇▅▄▅▄▄▄▃▂▃▂▂▁▁▁▁
epoch/precision,▁▃▂▄▃▅▅▄▅▆▅▇▇▇▇▇▇██▇
epoch/recall,▁▂▂▃▁▃▄▄▅▆▄▇▆▅▇▇███▆
epoch/true_negatives,▁▃▃▄▃▅▅▄▅▆▅▇▇▇▇▇▇██▇
